# AVI-DYS Time Series Classification

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import RocCurveDisplay

## Set User specific parameters

In [2]:
IDs = ['001', '003', '004', '005', '115', '116', '117', '118', '120', '121',
        '122', '123', '125', '126', '127', '128', '129', '130', '131', '132',
        '133', '135', '138', '151', '152', '301', '303', '304', '305', '306',
        '307', '308', '309']

data_path = "C:\\Users\\zacha\\Data\\AVI-DYS\\Results\\Post-Processing\\5-Concatenated"

## Format the DIS Scores

In [3]:
# Initialize an output DataFrame for the DIS scores (y)
cols = [
       'Dystonia-Duration', 'Dystonia-Amplitude',
       'Choreoathetosis-Duration', 'Choreoathetosis-Amplitude',
        ]

iterables = [IDs, ["11", "12"]]

index = pd.MultiIndex.from_product(iterables, names=["ID", "Side Code"])

y = pd.DataFrame(index=index, columns=cols)
y.reset_index(inplace=True)

In [4]:
# Put the DIS scores into a tidy format
df = pd.read_excel("C:\\Users\\zacha\\Repos\\AVI-DYS\\DIS-Included.xlsx")

for idx, i in enumerate(IDs):
    y['Dystonia-Duration'][idx*2] = df['Dystonia-Right-Duration-11'][idx]
    y['Dystonia-Duration'][idx*2+1] = df['Dystonia-Left-Duration-12'][idx]
    y['Dystonia-Amplitude'][idx*2] = df['Dystonia-Right-Amplitude-11'][idx]
    y['Dystonia-Amplitude'][idx*2+1] = df['Dystonia-Left-Amplitude-12'][idx]
    y['Choreoathetosis-Duration'][idx*2] = df['Choreoathetosis-Right-Duration-11'][idx]
    y['Choreoathetosis-Duration'][idx*2+1] = df['Choreoathetosis-Left-Duration-12'][idx]
    y['Choreoathetosis-Amplitude'][idx*2] = df['Choreoathetosis-Right-Amplitude-11'][idx]
    y['Choreoathetosis-Amplitude'][idx*2+1] = df['Choreoathetosis-Left-Amplitude-12'][idx]

## Create the 3D numpy array

In [5]:
# Initialize an output DataFrame for the Data (X)
cols = ['HLX_x', 'HLX_y', 'D1M_x', 'D1M_y', 'D3M_x', 'D3M_y', 'ANK_x',
       'ANK_y', 'KNE_x', 'KNE_y', 'KNE_ANK_length',
       'KNE_ANK_orientation', 'ANK_D3M_length', 'ANK_D3M_orientation',
       'ANK_D1M_length', 'ANK_D1M_orientation', 'D1M_HLX_length',
       'D1M_HLX_orientation']

df = pd.DataFrame(columns=cols)

In [6]:
# Flatten the DataFrame
for i in IDs:
   
    right_data = pd.read_csv(os.path.join(data_path,f'{i}-right-data.csv'))
    left_data = pd.read_csv(os.path.join(data_path,f'{i}-left-data.csv'))    
    
    right_flat = pd.DataFrame(index=[f'{i}-right'])
    for c in right_data.columns:
        right_flat[c] = [right_data[c].to_numpy()]  

    left_flat = pd.DataFrame(index=[f'{i}-left'])
    for c in left_data.columns:
        left_flat[c] = [left_data[c].to_numpy()]    
    
    df = pd.concat([df, right_flat, left_flat,])

# Extract the values from the DataFrame
data = df.values

# Create an empty 3D NumPy array
num_rows, num_columns = df.shape
num_elements = df.iloc[0, 0].shape[0]
X = np.empty((num_rows, num_columns, num_elements))

# Populate the 3D array with the values from the DataFrame
for i in range(num_rows):
    for j in range(num_columns):
        X[i, j] = data[i, j]

## HIVE-COTE v2.0

In [7]:
# Predict the Dystonia Duration Scores
Dystonia_Duration_y = y['Dystonia-Duration'].astype(str).to_numpy()

hc2 =HIVECOTEV2(
      stc_params=None,
      drcif_params=None,
      arsenal_params=None,
      tde_params=None,
      time_limit_in_minutes=1,
      save_component_probas=False,
      verbose=1,
      n_jobs=-1,
      random_state=42,
)
hc2.fit(X, Dystonia_Duration_y)
y_pred_Dystonia_Duration = hc2.predict(X)

STC  12:50:32 25/05/2023
STC train estimate  12:50:34 25/05/2023
STC weight = 0.3565372925346629
DrCIF  12:51:02 25/05/2023
DrCIF train estimate  12:51:05 25/05/2023
DrCIF weight = 0.3038145563658962
Arsenal  12:51:48 25/05/2023
Arsenal train estimate  12:51:51 25/05/2023
Arsenal weight = 0.3293853469160256


c:\Users\zacha\anaconda3\envs\ml\lib\site-packages\numba\typed\typeddict.py:33: NumbaTypeSafetyWarning: unsafe cast from int64 to uint32. Precision may be lost.
  d[key] = value


TDE  12:52:02 25/05/2023
TDE train estimate  12:52:02 25/05/2023
TDE weight = 0.13491623809680411


In [8]:
accuracy_score(Dystonia_Duration_y, y_pred_Dystonia_Duration)

1.0

In [9]:
confusion_matrix(Dystonia_Duration_y, y_pred_Dystonia_Duration)

array([[11,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0,  0,  3,  0,  0],
       [ 0,  0,  0,  6,  0],
       [ 0,  0,  0,  0, 45]], dtype=int64)

In [10]:
# Predict the Dyskinesis Amplitude Scores
Dystonia_Amplitude_y = y['Dystonia-Amplitude'].astype(str).to_numpy()

hc2 =HIVECOTEV2(
      stc_params=None,
      drcif_params=None,
      arsenal_params=None,
      tde_params=None,
      time_limit_in_minutes=1,
      save_component_probas=False,
      verbose=1,
      n_jobs=-1,
      random_state=42,
)
hc2.fit(X, Dystonia_Amplitude_y)
y_pred_Dystonia_Amplitude = hc2.predict(X)




STC  12:53:38 25/05/2023
STC train estimate  12:54:03 25/05/2023
STC weight = 0.09877138738583764
DrCIF  12:54:30 25/05/2023
DrCIF train estimate  12:54:34 25/05/2023
DrCIF weight = 0.08851854381531314
Arsenal  12:55:16 25/05/2023
Arsenal train estimate  12:55:18 25/05/2023
Arsenal weight = 0.16399153063315347
TDE  12:55:30 25/05/2023
TDE train estimate  12:55:30 25/05/2023
TDE weight = 0.0625


In [11]:
accuracy_score(Dystonia_Amplitude_y, y_pred_Dystonia_Amplitude)

1.0

In [12]:
confusion_matrix(Dystonia_Amplitude_y, y_pred_Dystonia_Amplitude)

array([[11,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0],
       [ 0,  0,  9,  0,  0],
       [ 0,  0,  0, 10,  0],
       [ 0,  0,  0,  0, 35]], dtype=int64)

In [13]:
# Predict the Choreoathetosis Duration Scores
Choreoathetosis_Duration_y = y['Choreoathetosis-Duration'].astype(str).to_numpy()

hc2 =HIVECOTEV2(
      stc_params=None,
      drcif_params=None,
      arsenal_params=None,
      tde_params=None,
      time_limit_in_minutes=1,
      save_component_probas=False,
      verbose=1,
      n_jobs=-1,
      random_state=42,
)
hc2.fit(X, Choreoathetosis_Duration_y)
y_pred_Choreoathetosis_Duration = hc2.predict(X)

STC  12:57:26 25/05/2023
STC train estimate  12:57:51 25/05/2023
STC weight = 0.003453855695278185
DrCIF  12:58:18 25/05/2023
DrCIF train estimate  12:58:23 25/05/2023
DrCIF weight = 0.003453855695278185
Arsenal  12:59:06 25/05/2023
Arsenal train estimate  12:59:08 25/05/2023
Arsenal weight = 0.010249470664572092
TDE  12:59:21 25/05/2023
TDE train estimate  12:59:21 25/05/2023
TDE weight = 0.010249470664572092


In [14]:
accuracy_score(Choreoathetosis_Duration_y, y_pred_Choreoathetosis_Duration)

1.0

In [15]:
confusion_matrix(Choreoathetosis_Duration_y, y_pred_Choreoathetosis_Duration)

array([[18,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0],
       [ 0,  0, 16,  0,  0],
       [ 0,  0,  0,  7,  0],
       [ 0,  0,  0,  0,  8]], dtype=int64)

In [16]:
# Predict the Choreoathetosis Amplitude Scores
Choreoathetosis_Amplitude_y = y['Choreoathetosis-Amplitude'].astype(str).to_numpy()

hc2 =HIVECOTEV2(
      stc_params=None,
      drcif_params=None,
      arsenal_params=None,
      tde_params=None,
      time_limit_in_minutes=1,
      save_component_probas=False,
      verbose=1,
      n_jobs=-1,
      random_state=42,
)
hc2.fit(X, Choreoathetosis_Amplitude_y)
y_pred_Choreoathetosis_Amplitude = hc2.predict(X)

STC  13:01:17 25/05/2023
STC train estimate  13:01:41 25/05/2023
STC weight = 0.010249470664572092
DrCIF  13:02:09 25/05/2023
DrCIF train estimate  13:02:13 25/05/2023
DrCIF weight = 0.010249470664572092
Arsenal  13:02:56 25/05/2023
Arsenal train estimate  13:02:58 25/05/2023
Arsenal weight = 0.0068681324472709415
TDE  13:03:10 25/05/2023
TDE train estimate  13:03:10 25/05/2023
TDE weight = 0.042688340960316914


In [17]:
accuracy_score(Choreoathetosis_Amplitude_y, y_pred_Choreoathetosis_Amplitude)

1.0

In [18]:
confusion_matrix(Choreoathetosis_Amplitude_y, y_pred_Choreoathetosis_Amplitude)

array([[18,  0,  0,  0,  0],
       [ 0, 15,  0,  0,  0],
       [ 0,  0, 21,  0,  0],
       [ 0,  0,  0,  9,  0],
       [ 0,  0,  0,  0,  3]], dtype=int64)